# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [2]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('C:\\Users\\Diego Alves\\Desktop\\Jupyter\\MLCourse\\emails\\spam', 'spam'))
data = data.append(dataFrameFromDirectory('C:\\Users\\Diego Alves\\Desktop\\Jupyter\\MLCourse\\emails\\ham', 'ham'))


Let's have a look at that DataFrame:

In [5]:
data

,class,message
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00006.5ab5620d3d7c6c0db76234556a16f6c1,spam,A POWERHOUSE GIFTING PROGRAM You Don't Want To...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00007.d8521faf753ff9ee989122f6816f87d7,spam,Help wanted. We are a 14 year old fortune 500...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00008.dfd941deb10f5eed78b1594b131c9266,spam,<html>\n\n<head>\n\n<title>ReliaQuote - Save U...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00009.027bf6e0b0c4ab34db3ce0ea4bf2edab,spam,TIRED OF THE BULL OUT THERE?\n\nWant To Stop L...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00010.445affef4c70feec58f9198cfbc22997,spam,"Dear ricardo1 ,\n\n\n\n<html>\n\n<body>\n\n<ce..."


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [4]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [4]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [6]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('C:\\Users\\Diego Alves\\Desktop\\Jupyter\\MLCourse\\emails\\spam', 'spam'))
data1 = data.append(dataFrameFromDirectory('C:\\Users\\Diego Alves\\Desktop\\Jupyter\\MLCourse\\emails\\ham', 'ham'))


In [7]:
data1

,class,message
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00006.5ab5620d3d7c6c0db76234556a16f6c1,spam,A POWERHOUSE GIFTING PROGRAM You Don't Want To...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00007.d8521faf753ff9ee989122f6816f87d7,spam,Help wanted. We are a 14 year old fortune 500...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00008.dfd941deb10f5eed78b1594b131c9266,spam,<html>\n\n<head>\n\n<title>ReliaQuote - Save U...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00009.027bf6e0b0c4ab34db3ce0ea4bf2edab,spam,TIRED OF THE BULL OUT THERE?\n\nWant To Stop L...
C:\Users\Diego Alves\Desktop\Jupyter\MLCourse\emails\spam\00010.445affef4c70feec58f9198cfbc22997,spam,"Dear ricardo1 ,\n\n\n\n<html>\n\n<body>\n\n<ce..."


## Split the data: 80% and 20%
Now we'll split the data in two - 80% of it will be used for "training" our model, and the other 20% for testing it. This way we can avoid overfitting.

In [8]:
x = data1['class']
y = data1['message']

In [11]:
from sklearn.model_selection import train_test_split
class_train, class_test, message_train, message_test= train_test_split(x,y,test_size=0.20)

In [12]:
print(class_train.shape,class_test.shape,message_train.shape,message_test.shape)

((2400L,), (600L,), (2400L,), (600L,))


## Lets try with the train data first(80%).

In [20]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(message_train.values)

classifier = MultinomialNB()
targets = class_train.values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='|S4')

## Lets try with the test data first(20%).

In [22]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(message_test.values)

classifier = MultinomialNB()
targets = class_test.values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['ham', 'ham'], dtype='|S4')